In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType
import pandas as pd
import statsmodels.api as sm

spark = SparkSession.builder.appName("Vehicles_PySpark_Analysis").getOrCreate()

# Cleaned data ko load karna (CSV se ya yahan dobara cleaning karne ke baad)
# Is analysis ke liye hum clean data ko dobara load karte hain
df_clean = spark.read.csv("/content/craigslist_data_for_power_bi_FINAL_TEXT_2.csv", header=True, inferSchema=True)
print(f"Analysis data loaded: {df_clean.count():,}")

In [ ]:
from pyspark.sql.types import IntegerType

# --- Feature Engineering: Age Calculation and Cleaning ---

# 1. Age Calculate Karna (PySpark)
# Humne pichle steps mein 2025 use kiya tha, wahi rakh rahe hain.
df_clean = df_clean.withColumn('Age', F.lit(2025).cast(IntegerType()) - F.col('year').cast(IntegerType()))

# 2. Impossible Age Values (Negative/Zero) Filter Karna (PySpark)
# Agar koi galti se future year (e.g., 2030) daal de, toh Age negative ho jaayegi. Use filter karna.
df_clean = df_clean.filter(F.col('Age') >= 0)

print(" Age calculation complete and impossible age values filtered.")
print(f"Rows remaining after age check: {df_clean.count():,}")

In [ ]:
# --- Pandas UDF Schema Definition ---
schema = StructType([
    StructField("manufacturer", StringType(), True),
    StructField("model", StringType(), True),
    StructField("Model_Count", LongType(), True),
    StructField("Est_Loss_Per_10k_Miles_USD", DoubleType(), True),
    StructField("Model_Reliability_R2", DoubleType(), True)
])

# --- Pandas UDF Definition (Model Run per Group) ---
@F.pandas_udf(schema, functionType=F.PandasUDFType.GROUPED_MAP)
def calculate_depreciation_udf(pandas_df):
    if len(pandas_df) < 50:
        return pd.DataFrame()

    # Statsmodels OLS (Pandas mein chalta hai, lekin Spark framework ke andar)
    try:
        X = sm.add_constant(pandas_df['odometer'])
        y = pandas_df['price']
        model = sm.OLS(y, X).fit()

        depreciation_rate = model.params['odometer']

        result = pd.DataFrame({
            'manufacturer': [pandas_df['manufacturer'].iloc[0]],
            'model': [pandas_df['model'].iloc[0]],
            'Model_Count': [len(pandas_df)],
            'Est_Loss_Per_10k_Miles_USD': [depreciation_rate * 10000 * -1], # Positive Loss
            'Model_Reliability_R2': [model.rsquared]
        })
        return result
    except:
        return pd.DataFrame()

# --- Grouped Map Apply Karna (Distributed Analysis) ---
master_depreciation_results_spark = df_clean.groupby("manufacturer", "model").apply(calculate_depreciation_udf)

# --- Final Save for Power BI ---
master_depreciation_results_spark.coalesce(1).write.csv(
    "reports/MASTER_DEPRECIATION_RATES.csv",
    mode="overwrite",
    header=True
)

print("\n MASTER_DEPRECIATION_RATES.csv file successfully created (Power BI Metrics Table).")

In [ ]:
# Overall depreciation ke liye data ko Pandas mein le aao (ek baar mein)
import numpy as np
df_all_data = df_clean.select("price", "odometer").toPandas()

# --- CLEANING STEP (most important) ---
df_all_data = df_all_data.replace([np.inf, -np.inf], np.nan)
df_all_data = df_all_data.dropna(subset=['price', 'odometer'])

# OLS Regression chalaana
X = sm.add_constant(df_all_data['odometer'])
y = df_all_data['price']

model_overall = sm.OLS(y, X).fit()

depreciation_per_mile = model_overall.params['odometer']
loss_per_10k_miles = depreciation_per_mile * 10000 * -1

# Final Pandas DataFrame bana kar save karna
results_df = pd.DataFrame({
    'Metric': ['Overall_Loss_Per_10k_Miles'],
    'Value': [loss_per_10k_miles],
    'Unit': ['USD']
})

results_df.to_csv('reports/analysis_key_metrics.csv', index=False)
print(" analysis_key_metrics.csv saved.")


In [ ]:
# --- Visualization Angle 1: Price vs. Odometer ---
print(" Visualizing Angle 1: Price vs. Odometer (Depreciation Trend)")

# PySpark se data ka 1% sample nikalna (tez plot ke liye)
df_sample = df_clean.select("price", "odometer").sample(False, 0.01, seed=42)
df_viz = df_sample.toPandas() # PySpark to Pandas conversion

plt.figure(figsize=(10, 6))
sns.regplot(
    data=df_viz,
    x='odometer',
    y='price',
    scatter_kws={'alpha': 0.1, 's': 10}, # Scatter points ko halka karna
    line_kws={'color': 'red', 'lw': 2}   # Regression line
)

plt.title('Price vs. Odometer Reading (Depreciation Trend)', fontsize=14)
plt.xlabel('Odometer Reading (Miles)')
plt.ylabel('Price (USD)')
plt.grid(alpha=0.3)
plt.show()

In [ ]:
# --- Visualization Angle 2: Car Type Distribution ---
print("\n Visualizing Angle 2: Top Car Types Ki Distribution")

# PySpark mein Aggregation
df_type_counts = df_clean.groupBy('type').count()
df_type_counts = df_type_counts.orderBy(F.desc('count')).limit(10) # Top 10 types

# PySpark to Pandas for Plotting
df_viz_type = df_type_counts.toPandas()

plt.figure(figsize=(10, 6))
sns.barplot(
    x='count',
    y='type',
    data=df_viz_type,
    palette='viridis'
)

plt.title('Top 10 Car Types by Count', fontsize=14)
plt.xlabel('Listing Count (Lakhs Mein)')
plt.ylabel('Car Type (Original Text)')
plt.show()

In [ ]:
# --- Visualization Angle 3: Average Price by Year ---
print("\n Visualizing Angle 3: Average Price by Year")

# PySpark mein Aggregation (Average Price nikalna)
df_avg_price = df_clean.groupBy('year').agg(
    F.round(F.mean('price'), 0).alias('Average_Price')
)
df_avg_price = df_avg_price.filter(F.col('year') > 1990) # Purani gaadiyon ko hatana
df_avg_price = df_avg_price.orderBy('year')

# PySpark to Pandas for Plotting
df_viz_year = df_avg_price.toPandas()

plt.figure(figsize=(12, 6))
sns.lineplot(
    x='year',
    y='Average_Price',
    data=df_viz_year,
    marker='o',
    color='darkblue'
)

plt.title('Average Car Price Trend Over Time (1990+)', fontsize=14)
plt.xlabel('Year of Manufacture')
plt.ylabel('Average Price (USD)')
plt.grid(alpha=0.3)
plt.show()

In [ ]:
# --- Visualization Angle 4: Transmission Distribution ---
print("\n Visualizing Angle 4: Transmission Distribution")

# PySpark mein Aggregation
df_trans_counts = df_clean.groupBy('transmission').count()

# PySpark to Pandas for Plotting
df_viz_trans = df_trans_counts.toPandas()

plt.figure(figsize=(8, 5))
sns.barplot(
    x='transmission',
    y='count',
    data=df_viz_trans,
    palette='pastel'
)

plt.title('Distribution by Transmission Type', fontsize=14)
plt.xlabel('Transmission Type (Original Text)')
plt.ylabel('Listing Count')
plt.show()

In [ ]:
# --- Visualization Angle 5: Top Manufacturers by Depreciation Loss ---
print("\n Visualizing Angle 5: Top 10 Manufacturers by Lowest Monetary Loss")

# 1. Depreciation Metrics file load karna (jo humne pichle step mein banayi thi)
# Note: Yeh file choti hai, isliye hum ise Pandas se load kar rahe hain
try:
    df_depr_metrics = pd.read_csv('\content\reports\MASTER_DEPRECIATION_RATES.csv')
except FileNotFoundError:
    print(" Error: MASTER_DEPRECIATION_RATES.csv file not found. Please ensure it was created and copied to /reports folder.")
    # Agar file nahi mili, toh aage nahi badhenge
    # PySpark DataFrame ko Pandas mein convert karna (Grouped Data)
    # (Agar aapne pichle step mein Spark DataFrame ko memory mein rakha ho toh)
    # df_depr_metrics = master_depreciation_results_spark.toPandas()
    # Uske bajaye, hum sirf code dikhayenge.
    pass


if 'df_depr_metrics' in locals() and not df_depr_metrics.empty:
    # 2. Manufacturer level par average loss nikalna
    df_manu_loss = df_depr_metrics.groupby('manufacturer')['Est_Loss_Per_10k_Miles_USD'].mean().reset_index()

    # 3. Sabse kam depreciate hone waale (yani 'Loss' sabse kam ho) top 10 chuno
    df_manu_loss_sorted = df_manu_loss.sort_values(by='Est_Loss_Per_10k_Miles_USD', ascending=True).head(10)

    # 4. Plotting
    plt.figure(figsize=(10, 6))
    sns.barplot(
        x='Est_Loss_Per_10k_Miles_USD',
        y='manufacturer',
        data=df_manu_loss_sorted,
        palette='magma_r' # Reverse palette, taaki Lowest Loss (Best) dark ho
    )

    plt.title('Top 10 Manufacturers: Lowest Estimated Loss per 10,000 Miles', fontsize=14)
    plt.xlabel('Estimated Loss Per 10,000 Miles (USD)')
    plt.ylabel('Manufacturer')
    plt.grid(axis='x', alpha=0.3)
    plt.show()

# Is tarah se aapki visualization bhi complete ho jayegi.

In [ ]:
# --- Final Visualization: Numerical Correlation Heatmap ---
print("\n Final Visualization: Numerical Correlation Heatmap")

# 1. Zaroori columns chuno aur PySpark to Pandas conversion (Yeh data already clean hai)
# 'df_clean' aapka PySpark DataFrame hai
df_corr_pandas = df_clean.select("price", "year", "odometer").toPandas()

# 2. Correlation Matrix calculate karna (Pandas ka istemal karke)
correlation_matrix = df_corr_pandas[['price', 'year', 'odometer']].corr()

# 3. Heatmap plot karna (Seaborn)
plt.figure(figsize=(8, 6))
sns.heatmap(
    correlation_matrix,
    annot=True,  # Values ko display karna
    cmap='coolwarm', # Color scheme
    fmt=".2f",       # 2 decimal places tak
    linewidths=.5,
    linecolor='black'
)

plt.title('Correlation Heatmap: Price, Year, and Odometer', fontsize=14)
plt.show()

In [ ]:
# --- Visualization Angle 6: Average Price by Age and Type (Depreciation Analysis) ---
print("\n Visualizing Angle 6: Average Price by Vehicle Age and Type")

# 1. Age Calculate Karna (Assuming current year is 2025 for consistency)
# Note: 'df_clean' aapka PySpark DataFrame hai
df_clean = df_clean.withColumn('Age', F.lit(2025) - F.col('year'))

# 2. Age ko Categories mein Baantna (Pandas' pd.cut ki jagah PySpark's F.when)
df_clean = df_clean.withColumn('Age_Group',
    F.when(F.col('Age') <= 5, '0-5 Years (Newest)')
     .when((F.col('Age') > 5) & (F.col('Age') <= 10), '6-10 Years (Mid-Age)')
     .when((F.col('Age') > 10) & (F.col('Age') <= 20), '11-20 Years (Old)')
     .otherwise('20+ Years (Very Old)')
)

# 3. Filtering & Average Price Calculate Karna (PySpark GroupBy)
main_types = ['sedan', 'truck', 'suv', 'coupe', 'hatchback']

avg_price_data_spark = df_clean.filter(F.col('type').isin(main_types)) \
                                .groupBy('Age_Group', 'type') \
                                .agg(F.mean('price').alias('Average_Price'))

# 4. Visualization ke liye Pandas mein convert karna
avg_price_data = avg_price_data_spark.toPandas()

# 5. Age Grouping order theek karna taaki plot mein sahi sequence mein aaye
age_order = ['0-5 Years (Newest)', '6-10 Years (Mid-Age)', '11-20 Years (Old)', '20+ Years (Very Old)']
avg_price_data['Age_Group'] = pd.Categorical(avg_price_data['Age_Group'], categories=age_order, ordered=True)
avg_price_data = avg_price_data.sort_values('Age_Group')


# 6. Plotting (Seaborn)
plt.figure(figsize=(14, 7))
sns.barplot(
    data=avg_price_data,
    x='Age_Group',
    y='Average_Price',
    hue='type',
    palette='viridis'
)

plt.title('Average Price by Vehicle Age and Type (Depreciation Analysis)', fontsize=14)
plt.xlabel('Vehicle Age Group')
plt.ylabel('Average Price (USD)')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.5)
plt.legend(title='Vehicle Type')
plt.tight_layout()
plt.show()

In [ ]:
# --- Visualization Angle 7: Price vs. Odometer Segmentation by Type ---
print("\n Final Visualization: Price vs. Odometer Segmentation by Type")

# 1. Filtering (PySpark) - Zaroori types ko filter karna
main_types_for_odom = ['sedan', 'truck', 'suv', 'hatchback']
df_odom_spark = df_clean.filter(F.col('type').isin(main_types_for_odom))

# 2. Sampling (PySpark) - Massive data points ko handle karne ke liye 2% sample lena
df_odom_sample = df_odom_spark.select('odometer', 'price', 'type').sample(False, 0.02, seed=42)

# 3. PySpark to Pandas Conversion
df_odom = df_odom_sample.toPandas()

# 4. Visualization (Seaborn)
plt.figure(figsize=(12, 7))

# Odometer aur Price ka Scatter Plot (Type ke hisaab se)
sns.scatterplot(
    data=df_odom,
    x='odometer',
    y='price',
    hue='type',
    style='type',
    palette='deep',
    alpha=0.6,
    s=30 # Size of points
)

# X-axis limits ko theek karna (original code ke mutabik)
plt.xlim(0, 300000)
plt.ylim(0, 50000)

plt.title('Price vs. Odometer Reading, Segmented by Vehicle Type', fontsize=16)
plt.xlabel('Odometer Reading (Miles)', fontsize=12)
plt.ylabel('Price (USD)', fontsize=12)
plt.legend(title='Vehicle Type')
plt.grid(axis='y', alpha=0.3)
plt.show()

In [ ]:
# --- Visualization Angle 8: Median Price by Title Status ---
print("\n Final Visualization: Median Price by Title Status (Risk Devaluation)")

# 1. Filtering & Aggregation (PySpark) - Median Price nikalna
main_titles = ['clean', 'salvage', 'rebuilt', 'lien']
median_price_data_spark = df_clean.filter(F.col('title_status').isin(main_titles)) \
                                  .groupBy('title_status') \
                                  .agg(F.median('price').alias('Median_Price'))

# 2. PySpark to Pandas Conversion
median_price_data = median_price_data_spark.toPandas()

# 3. Sorting (Pandas) - Price ke hisaab se sort karna
median_price_data = median_price_data.sort_values(by='Median_Price', ascending=False)

# 4. Visualization (Seaborn)
plt.figure(figsize=(10, 6))
sns.barplot(
    data=median_price_data,
    x='title_status',
    y='Median_Price',
    palette='coolwarm'
)

plt.title('Median Price by Title Status (Risk Devaluation)', fontsize=16)
plt.xlabel('Title Status', fontsize=12)
plt.ylabel('Median Price (USD)', fontsize=12)
plt.grid(axis='y', alpha=0.3)
plt.show()

In [ ]:
# --- Final Analysis: Price vs. Age vs. Mileage Correlation ---

print("\n Final Analysis: Which Factor Affects Price More (Age vs. Mileage)?")

# 1. Zaroori numerical columns ko PySpark DataFrame se select karna
# Note: 'Age' column pichli visualization mein F.when se ban chuki hai.
numerical_features = ['price', 'odometer', 'Age']
df_corr_spark = df_clean.select(numerical_features)

# 2. PySpark to Pandas conversion for correlation calculation and plotting
df_corr = df_corr_spark.toPandas()

# 3. Correlation Matrix calculate karna (Pandas ka istemal karke)
correlation_matrix = df_corr.corr()

# 4. Heatmap plot karna (Seaborn)
plt.figure(figsize=(8, 7))
sns.heatmap(
    correlation_matrix,
    annot=True,
    cmap='coolwarm',
    fmt=".2f",
    linewidths=.5
)

plt.title('Correlation Heatmap: Which Factor Affects Price Most?', fontsize=16)
plt.show()

In [ ]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import functions as F

print("\n Overall Linear Regression: Depreciation Rate Calculation (PySpark MLlib)")

# 1. Outlier Filtering (PySpark equivalent of Z-score < 3)
# Mean aur Standard Deviation calculate karna
stats_df = df_clean.agg(
    F.mean("price").alias("price_mean"),
    F.stddev("price").alias("price_std"),
    F.mean("odometer").alias("odometer_mean"),
    F.stddev("odometer").alias("odometer_std")
).collect()[0]

# Filtering Conditions banana (3 standard deviations ke andar)
df_depr_filtered = df_clean.filter(
    (F.col("price") > stats_df["price_mean"] - 3 * stats_df["price_std"]) &
    (F.col("price") < stats_df["price_mean"] + 3 * stats_df["price_std"]) &
    (F.col("odometer") > stats_df["odometer_mean"] - 3 * stats_df["odometer_std"]) &
    (F.col("odometer") < stats_df["odometer_mean"] + 3 * stats_df["odometer_std"])
).select("price", "odometer")

print(f"Rows after 3-sigma outlier filtering: {df_depr_filtered.count():,}")

# 2. Vector Assembler: PySpark ML ke liye features ko vector mein jodna
assembler = VectorAssembler(inputCols=['odometer'], outputCol="features")
df_lr_ready = assembler.transform(df_depr_filtered)

# 3. Distributed Linear Regression Model Fit Karna (PySpark MLlib)
lr = LinearRegression(featuresCol="features", labelCol="price")
lr_model = lr.fit(df_lr_ready)

# 4. Result Nikalna: Coefficient (Depreciation Rate)
depreciation_per_mile = lr_model.coefficients[0]
loss_per_10k_miles = depreciation_per_mile * 10000

# Final Results Print Karna
print(f"\n Odometer Coefficient (Depreciation Rate per 1 Mile): ${depreciation_per_mile:,.4f}")
print(f" Loss Per 10,000 Miles: ${loss_per_10k_miles:,.2f}")

In [ ]:
from pyspark.sql import functions as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

print("\n Visualization: Price vs. Odometer Regression Trend (Final Visual)")

# 1. Outlier Filtering (PySpark Z-score filtering logic)
# PySpark mein Z-score filtering ke liye Mean aur Standard Deviation dobara calculate karna
stats_df = df_clean.agg(
    F.mean("price").alias("price_mean"),
    F.stddev("price").alias("price_std"),
    F.mean("odometer").alias("odometer_mean"),
    F.stddev("odometer").alias("odometer_std")
).collect()[0]

# Filtering Conditions banana (3 standard deviations ke andar)
df_depr_filtered_spark = df_clean.filter(
    (F.col("price") > stats_df["price_mean"] - 3 * stats_df["price_std"]) &
    (F.col("price") < stats_df["price_mean"] + 3 * stats_df["price_std"]) &
    (F.col("odometer") > stats_df["odometer_mean"] - 3 * stats_df["odometer_std"]) &
    (F.col("odometer") < stats_df["odometer_mean"] + 3 * stats_df["odometer_std"])
).select("price", "odometer")

# 2. Sampling (PySpark) - Plotting ke liye 2% sample lena
df_depr_sample = df_depr_filtered_spark.sample(False, 0.02, seed=42)

# 3. PySpark to Pandas Conversion
df_depr = df_depr_sample.toPandas()

# 4. Visualization (Seaborn regplot)
plt.figure(figsize=(12, 7))

# 'regplot' automatically scatter plot banata hai aur uspar linear trend line fit karta hai.
sns.regplot(
    data=df_depr,
    x='odometer',
    y='price',
    scatter_kws={'alpha': 0.3, 's': 10}, # Points ko halka aur chota karna
    line_kws={'color': 'red', 'lw': 3}   # Trend line ko mota aur laal karna
)

# Limits set karna (taaki plot zyada clear ho)
plt.xlim(0, 300000)
plt.ylim(0, 50000)

plt.title('Car Price Depreciation vs. Odometer Reading (Mileage)', fontsize=16)
plt.xlabel('Odometer Reading (Miles)', fontsize=12)
plt.ylabel('Price (USD)', fontsize=12)
plt.grid(axis='y', alpha=0.3)
plt.show()

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType
import pandas as pd
import statsmodels.api as sm
import numpy as np
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

print("\n Starting Comprehensive Distributed Analysis (Master Metrics & Grouped Regression)")

# ====================================================================
# PART A: OVERALL KPI CALCULATION (PySpark MLlib & Save)
# ====================================================================

# 1. PySpark MLlib: Linear Regression (Z-score filtering ke baad)
# (Assumes df_clean already has the data after all previous steps)

# Filtering (3-sigma Outlier Removal)
stats_df = df_clean.agg(F.mean("price").alias("price_mean"),
                        F.stddev("price").alias("price_std"),
                        F.mean("odometer").alias("odometer_mean"),
                        F.stddev("odometer").alias("odometer_std")).collect()[0]

df_depr_filtered = df_clean.filter(
    (F.col("price") > stats_df["price_mean"] - 3 * stats_df["price_std"]) &
    (F.col("price") < stats_df["price_mean"] + 3 * stats_df["price_std"]) &
    (F.col("odometer") > stats_df["odometer_mean"] - 3 * stats_df["odometer_std"]) &
    (F.col("odometer") < stats_df["odometer_mean"] + 3 * stats_df["odometer_std"])
).select("price", "odometer")

assembler = VectorAssembler(inputCols=['odometer'], outputCol="features")
df_lr_ready = assembler.transform(df_depr_filtered)

lr = LinearRegression(featuresCol="features", labelCol="price")
lr_model = lr.fit(df_lr_ready)
depreciation_per_mile = lr_model.coefficients[0]
loss_per_10k_miles = depreciation_per_mile * 10000

# 2. Key Metric DataFrame Banana aur Save Karna
results_df = pd.DataFrame({
    'Metric': ['Overall Depreciation per 1 Mile', 'Overall Loss Per 10k Miles'],
    'Value': [depreciation_per_mile, loss_per_10k_miles],
    'Unit': ['USD', 'USD']
})

results_df.to_csv('reports/analysis_key_metrics.csv', index=False)
print(" analysis_key_metrics.csv file saved (Overall KPI).")


# ====================================================================
# PART B: GROUPED REGRESSION (MASTER DEPRECIATION TABLE)
# ====================================================================

# 1. Pandas UDF Schema Definition
schema = StructType([
    StructField("manufacturer", StringType(), True),
    StructField("model", StringType(), True),
    StructField("Model_Count", LongType(), True),
    StructField("Depreciation_per_Mile", DoubleType(), True),
    StructField("Loss_Per_10k_Miles", DoubleType(), True),
    StructField("R2_Score", DoubleType(), True)
])

# 2. Pandas UDF Definition (OLS Model Run per Group)
@F.pandas_udf(schema, functionType=F.PandasUDFType.GROUPED_MAP)
def calculate_depreciation_udf(pandas_df):
    """Har (manufacturer, model) group ke liye OLS regression chalaana."""
    # Kam se kam 50 listings zaroori hain reliable calculation ke liye (jaisa ki aapne manga)
    if len(pandas_df) < 50:
        return pd.DataFrame()

    try:
        X = sm.add_constant(pandas_df['odometer'])
        y = pandas_df['price']
        model = sm.OLS(y, X).fit()

        depreciation_rate = model.params['odometer']

        # Result Pandas Series banana
        result = pd.Series({
            'manufacturer': pandas_df['manufacturer'].iloc[0],
            'model': pandas_df['model'].iloc[0],
            'Model_Count': len(pandas_df),
            'Depreciation_per_Mile': depreciation_rate,
            'Loss_Per_10k_Miles': depreciation_rate * 10000,
            'R2_Score': model.rsquared
        })
        return result.to_frame().T
    except:
        return pd.DataFrame()

# 3. Grouped Map Apply Karna (Distributed Analysis)
master_depreciation_results_spark = df_clean.groupby("manufacturer", "model").apply(calculate_depreciation_udf)

# 4. Final Cleanup aur Save Karna (Master Table)
# Depreciation Rate ko positive loss mein badalna
master_depreciation_results_spark = master_depreciation_results_spark.withColumn(
    'Est_Loss_Per_10k_Miles_USD', F.col('Loss_Per_10k_Miles') * -1)
master_depreciation_results_spark = master_depreciation_results_spark.withColumn(
    'Model_Reliability_R2', F.round(F.col('R2_Score'), 3))


# Final columns chunna aur save karna (jaisa ki aapke original code mein tha)
df_final_master = master_depreciation_results_spark.select(
    "manufacturer",
    "model",
    "Model_Count",
    "Est_Loss_Per_10k_Miles_USD",
    "Model_Reliability_R2"
)

df_final_master.coalesce(1).write.csv(
    "reports/MASTER_DEPRECIATION_RATES.csv",
    mode="overwrite",
    header=True
)

print(" MASTER_DEPRECIATION_RATES.csv file successfully created (Power BI Master Table).")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

print("\n Final Visualization: Top 10 Models by Lowest Depreciation Loss")

# 1. Master Depreciation File Load Karna (Pandas mein, kyunki yeh chota final metrics file hai)
df_master_depr = pd.read_csv('/content/reports/MASTER_DEPRECIATION_RATES.csv/part-00000-e1d4306c-8b4d-48b8-97ab-a11a05dd1b7e-c000.csv')

# 2. Results ko Filter aur Sort Karna
# Acche results ke liye, sirf woh models chunein jinki reliability (R2) 0.5 se zyada hai.
df_reliable_depr = df_master_depr[df_master_depr['Model_Reliability_R2'] > 0.5]

# Loss sabse kam ho (Lowest Loss = Best Value Retention)
df_top_10_low_loss = df_reliable_depr.sort_values(
    by='Est_Loss_Per_10k_Miles_USD',
    ascending=True
).head(10)

# 'manufacturer' aur 'model' ko jodkar label banana
df_top_10_low_loss['Model_Label'] = df_top_10_low_loss['manufacturer'] + ' ' + df_top_10_low_loss['model']

# 3. Visualization (Bar Plot)
plt.figure(figsize=(10, 7))

sns.barplot(
    data=df_top_10_low_loss,
    x='Est_Loss_Per_10k_Miles_USD',
    y='Model_Label',
    palette='viridis_r' # Reverse palette taaki "Best" models (Lowest Loss) dark green dikhein
)

plt.title('Top 10 Vehicles by Lowest Estimated Loss per 10k Miles (R-squared > 0.5)', fontsize=14)
plt.xlabel('Estimated Loss Per 10,000 Miles (USD)')
plt.ylabel('Vehicle Model')
plt.grid(axis='x', alpha=0.3)
plt.show()